In [1]:
import pandas as pd
import numpy as np
import panel as pn
import folium as fm
import param
from branca.element import Template, MacroElement
pn.extension('tabulator')
pn.extension()
pn.extension(sizing_mode="stretch_width")
import hvplot.pandas
import plotly.express as px

## (1) Define Panel Widgets

In [2]:
# Define Panel widgets
week_slider = pn.widgets.IntSlider(name='Week slider', start=1, end=52, step=1, value=9)
#week_slider

In [3]:
# Define Panel widgets
confidence_slider = pn.widgets.IntSlider(name='Confidence slider', start=50, end=100, step=1, value=75)
confidence_slider

IntSlider(end=100, name='Confidence slider', sizing_mode='stretch_width', start=50, value=75)

In [4]:
confidence_range_slider = pn.widgets.IntRangeSlider(name='Confidence',start=50, end=100, value=(50, 100), step=1)

confidence_range_slider

IntRangeSlider(end=100, name='Confidence', sizing_mode='stretch_width', start=50, value=(50, 100), value_end=100, value_start=50)

In [5]:
confidence_range_slider.value[0]

50

In [6]:
week_range_slider = pn.widgets.IntRangeSlider(name='Week',start=9, end=52, value=(11, 15), step=1)

week_range_slider

IntRangeSlider(end=52, name='Week', sizing_mode='stretch_width', start=9, value=(11, 15), value_end=15, value_start=11)

In [7]:
week_range_slider.value[1]

15

## (2) Reading Data

In [8]:
#dfm = pd.read_csv('wildfire-data/active-data/ts_active_fire_shape_24h.csv')

In [9]:
#dfm = pd.read_csv('https://raw.githubusercontent.com/oneon/firedataset/main/ts_active_fire_shape.csv')

In [10]:
# cache data to improve dashboard performance
if 'datamap' not in pn.state.cache.keys():

    dfm = pd.read_csv('https://raw.githubusercontent.com/oneon/firedataset/main/ts_active_fire_shape.csv')

    pn.state.cache['datamap'] = dfm.copy()

else: 

    dfm = pn.state.cache['datamap']

In [11]:
# cache data to improve dashboard performance
if 'datapre' not in pn.state.cache.keys():

    dfp = pd.read_csv('https://raw.githubusercontent.com/oneon/firedataset/main/ts_predict_fire_shape.csv')

    pn.state.cache['datapre'] = dfp.copy()

else: 

    dfp = pn.state.cache['datapre']

In [12]:
# cache data to improve dashboard performance
if 'datahis' not in pn.state.cache.keys():

    dfh = pd.read_csv('https://raw.githubusercontent.com/oneon/firedataset/main/ts_yearly_fire.csv')

    pn.state.cache['datahis'] = dfh.copy()

else: 

    dfh = pn.state.cache['datahis']

In [13]:
dfm = dfm[dfm['Dist_Name'].str.contains('Nalgonda') | dfm['Dist_Name'].str.contains('Nagarkurnool')]

In [14]:
dfp = dfp[dfp['Dist_Name'].str.contains('Nalgonda') | dfp['Dist_Name'].str.contains('Nagarkurnool')]

In [15]:
#dfp

In [16]:
dfm.shape, dfp.shape

((738, 13), (373, 13))

In [17]:
# Make DataFrame Pipeline Interactive
idfm = dfm.interactive()

In [18]:
idfm.shape

In [19]:
yaxis_duration = pn.widgets.RadioButtonGroup(
    name='Y axis', 
    options=['24h', '48h', '7d'], 
    button_type='success'
)
yaxis_duration

map_data_pipeline = (
    idfm[
        (idfm.duration == yaxis_duration)
    ]
    .groupby(['latitude', 'longitude', 'acq_date', 'acq_time', 'Mandal_Nam', 'Dist_Name', 'duration', 'instrument', 'week']).confidence.max()
    .to_frame()
    .reset_index()  
    .reset_index(drop=True)
)

In [20]:
#map_data_pipeline.columns

In [21]:
yaxis_duration.value

'24h'

## (3) Table - Firedata 

In [22]:
active_fire_table = map_data_pipeline.pipe(pn.widgets.Tabulator, pagination='remote', page_size = 20, sizing_mode='stretch_width') 
#active_fire_table

In [23]:
#map_data_pipeline.sample(10)

In [24]:
def get_map(lat=16.385318, long=78.9740939, zoom_start=9):
    return fm.Map(location=[lat,long], zoom_start=zoom_start)

map = get_map()

#pn.panel(map, height=400)

## (4) Data Active - Firedata 

In [25]:
def get_df_aqi(confidence=50):
    aqi = dfm[dfm['duration'] == yaxis_duration.value]
    aqi = aqi[aqi['confidence'] >= confidence]
    return pd.DataFrame(aqi)
    #return aqi

df_aqi = get_df_aqi()

## (5) Data Forecast - Firedata 

In [26]:
def get_dfp_aqi(confidence=50):
    aqi = dfp[dfp['week'] > week_range_slider.value[0]] 
    aqi = aqi[aqi['week'] < week_range_slider.value[1]]
    aqi = aqi[aqi['confidence'] >= confidence]
    return pd.DataFrame(aqi)
    #return aqi

df_aqii = get_dfp_aqi()

In [27]:
df_aqi.shape, df_aqii.shape

((84, 13), (11, 13))

In [28]:
def add_aqi_circles(map, df_aqi):
    yellow_p1  = fm.map.FeatureGroup()
    orange_p1 = fm.map.FeatureGroup()
    red_p1    = fm.map.FeatureGroup()
    
    for _, row in df_aqi.iterrows():
          
    
        html=f"""
            <h4> {row.Dist_Name}</h4>
            <p>Mandal {row.Mandal_Nam}</p>
            <ul>
                <li>Date {row.acq_date}</li>
                <li>Time {row.acq_time}</li>
                <li>Lat {row.latitude}</li>
                <li>Long {row.longitude}</li>
                <li>Confidence {row.confidence}</li>
            </ul>
            </p>
            """
        iframe = fm.IFrame(html=html, width=200, height=200)
        popup = fm.Popup(iframe, max_width=2650)
    
    
        if (row.confidence >= 50) and (row.confidence < 60):
            feature_group = yellow_p1
            customicon = 'fire-flame-yellow.png'
        elif (row.confidence >= 60) and (row.confidence < 80):
            feature_group = orange_p1
            customicon = 'fire-flame-orange.png'
        elif (row.confidence >= 80) and (row.confidence <= 100):
            feature_group = red_p1
            customicon = 'fire-flame-red.png'
        else:
            feature_group = yellow_p1
            customicon = 'fire-flame-yellow.png'
            
            
        pushpin = fm.features.CustomIcon(customicon, icon_size=(30,30))
        feature_group.add_child(
            fm.Marker(
                [row.latitude, row.longitude],
                icon=pushpin,
                popup=popup
            )
        )

    map.add_child(yellow_p1)
    map.add_child(orange_p1)
    map.add_child(red_p1)

    template = """
    {% macro html(this, kwargs) %}

    <!doctype html>
    <html lang="en">
    <head>
      <meta charset="utf-8">
      <meta name="viewport" content="width=device-width, initial-scale=1">
      <title>jQuery UI Draggable - Default functionality</title>
      <link rel="stylesheet" href="//code.jquery.com/ui/1.12.1/themes/base/jquery-ui.css">

      <script src="https://code.jquery.com/jquery-1.12.4.js"></script>
      <script src="https://code.jquery.com/ui/1.12.1/jquery-ui.js"></script>

      <script>
      $( function() {
        $( "#maplegend" ).draggable({
                        start: function (event, ui) {
                            $(this).css({
                                right: "auto",
                                top: "auto",
                                bottom: "auto"
                            });
                        }
                    });
    });

      </script>
    </head>
    <body>


    <div id='maplegend' class='maplegend' 
        style='position: absolute; z-index:9999; border:2px solid grey; background-color:rgba(255, 255, 255, 0.8);
         border-radius:6px; padding: 10px; font-size:14px; right: 20px; bottom: 20px;'>

    <div class='legend-title'>Legend Confidence</div>
    <div class='legend-scale'>
      <ul class='legend-labels'>
        <li><span style='background:yellow;opacity:0.7;'></span>50 to 60</li>
        <li><span style='background:orange;opacity:0.7;'></span>60 to 80</li>
        <li><span style='background:red;opacity:0.7;'></span>80 to 100</li>

      </ul>
    </div>
    </div>

    </body>
    </html>

    <style type='text/css'>
      .maplegend .legend-title {
        text-align: left;
        margin-bottom: 5px;
        font-weight: bold;
        font-size: 90%;
        }
      .maplegend .legend-scale ul {
        margin: 0;
        margin-bottom: 5px;
        padding: 0;
        float: left;
        list-style: none;
        }
      .maplegend .legend-scale ul li {
        font-size: 80%;
        list-style: none;
        margin-left: 0;
        line-height: 18px;
        margin-bottom: 2px;
        }
      .maplegend ul.legend-labels li span {
        display: block;
        float: left;
        height: 16px;
        width: 30px;
        margin-right: 5px;
        margin-left: 0;
        border: 1px solid #999;
        }
      .maplegend .legend-source {
        font-size: 80%;
        color: #777;
        clear: both;
        }
      .maplegend a {
        color: #777;
        }
    </style>
    {% endmacro %}"""

    macro = MacroElement()
    macro._template = Template(template)

    map.add_child(macro)
    
#add_aqi_circles(map, df_aqi)
#.panel(map, height=400)

In [29]:
def add_aqi_predict_circles(map, df_aqi):
    orange_p1  = fm.map.FeatureGroup()
    red_p1 = fm.map.FeatureGroup()
    yellow_p1 = fm.map.FeatureGroup()

    for _, row in df_aqi.iterrows():
        
    
        html=f"""
            <h4> {row.Dist_Name}</h4>
            <p>Mandal {row.Mandal_Nam}</p>
            <ul>
                <li>Week {row.week}</li>
                <li>Lat {row.latitude}</li>
                <li>Long {row.longitude}</li>
                <li>Confidence {row.confidence}</li>
            </ul>
            </p>
            """
        iframe = fm.IFrame(html=html, width=200, height=200)
        popup = fm.Popup(iframe, max_width=2650)
        
        if (row.confidence >= 50) and (row.confidence < 60):
            feature_group = yellow_p1
            customicon = 'fire-flame-yellow.png'
        elif (row.confidence >= 60) and (row.confidence < 80):
            feature_group = orange_p1
            customicon = 'fire-flame-orange.png'
        elif (row.confidence >= 80) and (row.confidence <= 100):
            feature_group = red_p1
            customicon = 'fire-flame-red.png'
        else:
            feature_group = yellow_p1
            customicon = 'fire-flame-yellow.png'
    
        pushpin = fm.features.CustomIcon(customicon, icon_size=(30,30))
        feature_group.add_child(
            fm.Marker(
                [row.latitude, row.longitude],
                icon=pushpin,
                popup=popup
            )
        )

    map.add_child(orange_p1)
    map.add_child(red_p1)
    map.add_child(yellow_p1)
    
    template = """
    {% macro html(this, kwargs) %}

    <!doctype html>
    <html lang="en">
    <head>
      <meta charset="utf-8">
      <meta name="viewport" content="width=device-width, initial-scale=1">
      <title>jQuery UI Draggable - Default functionality</title>
      <link rel="stylesheet" href="//code.jquery.com/ui/1.12.1/themes/base/jquery-ui.css">

      <script src="https://code.jquery.com/jquery-1.12.4.js"></script>
      <script src="https://code.jquery.com/ui/1.12.1/jquery-ui.js"></script>

      <script>
      $( function() {
        $( "#maplegend" ).draggable({
                        start: function (event, ui) {
                            $(this).css({
                                right: "auto",
                                top: "auto",
                                bottom: "auto"
                            });
                        }
                    });
    });

      </script>
    </head>
    <body>


    <div id='maplegend' class='maplegend' 
        style='position: absolute; z-index:9999; border:2px solid grey; background-color:rgba(255, 255, 255, 0.8);
         border-radius:6px; padding: 10px; font-size:14px; right: 20px; bottom: 20px;'>

    <div class='legend-title'>Legend Confidence</div>
    <div class='legend-scale'>
      <ul class='legend-labels'>
        <li><span style='background:yellow;opacity:0.7;'></span>50 to 60</li>
        <li><span style='background:orange;opacity:0.7;'></span>60 to 80</li>
        <li><span style='background:red;opacity:0.7;'></span>80 to 100</li>

      </ul>
    </div>
    </div>

    </body>
    </html>

    <style type='text/css'>
      .maplegend .legend-title {
        text-align: left;
        margin-bottom: 5px;
        font-weight: bold;
        font-size: 90%;
        }
      .maplegend .legend-scale ul {
        margin: 0;
        margin-bottom: 5px;
        padding: 0;
        float: left;
        list-style: none;
        }
      .maplegend .legend-scale ul li {
        font-size: 80%;
        list-style: none;
        margin-left: 0;
        line-height: 18px;
        margin-bottom: 2px;
        }
      .maplegend ul.legend-labels li span {
        display: block;
        float: left;
        height: 16px;
        width: 30px;
        margin-right: 5px;
        margin-left: 0;
        border: 1px solid #999;
        }
      .maplegend .legend-source {
        font-size: 80%;
        color: #777;
        clear: both;
        }
      .maplegend a {
        color: #777;
        }
    </style>
    {% endmacro %}"""

    macro = MacroElement()
    macro._template = Template(template)

    map.add_child(macro)
    
#add_aqi_predict_circles(map, df_aqii)
#pn.panel(map, height=400)

## (6) Create Map Active Fire 

In [30]:
class PanelFoliumMap(param.Parameterized):
    Confidence = param.Integer(50, bounds=(50,100))
        
    def __init__(self, **params):
        super().__init__(**params)
        self.map = get_map()
        self.folium_pane = pn.pane.plot.Folium(sizing_mode="stretch_both", min_height=500, margin=0)    
        self.view = pn.Column(
            self.param.Confidence,
            self.folium_pane,
            sizing_mode="stretch_both", height=500
        )
        self._update_map()

    @param.depends("Confidence", watch=True)
    def _update_map(self):
        self.map = get_map()
        df_aqi = get_df_aqi(confidence=self.Confidence)
        add_aqi_circles(self.map, df_aqi)
        self.folium_pane.object = self.map

        
#app = PanelFoliumMap()
#app.view

## (7) Bar chart with Forecast data

In [31]:
#dfp

In [32]:
# Make DataFrame Pipeline Interactive
idfp = dfp.interactive()

In [33]:
#dfp

In [34]:
# Radio buttons for District Name
yaxis_district = pn.widgets.RadioButtonGroup(
    name='Y axis', 
    options=['Nagarkurnool', 'Nalgonda',],
    button_type='success'
)
week_predict_bar_pipeline = (
    idfp[
        (idfp.Dist_Name == yaxis_district)       
    ]
    .groupby(['week']).size()
    .to_frame()
    .reset_index()
    .reset_index(drop=True)
)

In [35]:
#week_predict_bar_pipeline

In [36]:
week_predict_bar_pipeline.columns = ['week', 'Dist_Name', 'fire_count']

In [37]:
#week_predict_bar_pipeline

In [38]:
week_predict_bar_plot = week_predict_bar_pipeline.hvplot(kind='bar', 
                                                     x='week', 
                                                     y='0', 
                                                     title='Weekly fire count forecast')
#week_predict_bar_plot

## (8) Week vs Confidence scatterplot

In [39]:
week_vs_conf_scatterplot_pipeline = (
    idfp
    .groupby(['latitude', 'longitude', 'week', 'Dist_Name'])['confidence'].mean()
    .to_frame()
    .reset_index()
    .reset_index(drop=True)
)

In [40]:
week_vs_conf_scatterplot = week_vs_conf_scatterplot_pipeline.hvplot(x='week', 
                                                                y='confidence', 
                                                                by='Dist_Name', 
                                                                size=80, kind="scatter", 
                                                                alpha=0.7,
                                                                legend=False, 
                                                                height=500, 
                                                                width=500)
#week_vs_conf_scatterplot

## (9) Table - Firedata Forecast 

In [41]:
week_predict_pipeline = (
    idfp[
        (idfp.week == week_slider)
    ].groupby(['latitude', 'longitude', 'week', 'Dist_Name'])['confidence'].mean()
    .reset_index()
    .reset_index(drop=True)
)

In [42]:
#week_predict_pipeline

In [43]:
predict_table = week_predict_pipeline.pipe(pn.widgets.Tabulator, pagination='remote', page_size = 20, sizing_mode='stretch_width') 
#predict_table

## (10) Create Map Forecast 

In [44]:
class PanelFoliumMapForecast(param.Parameterized):
    Confidence = param.Integer(50, bounds=(50,100))
        
    def __init__(self, **params):
        super().__init__(**params)
        self.map = get_map()
        self.folium_pane = pn.pane.plot.Folium(sizing_mode="stretch_both", min_height=500, margin=0)    
        self.view = pn.Column(
            self.param.Confidence,
            self.folium_pane,
            sizing_mode="stretch_both", height=500
        )
        self._update_map()

    @param.depends("Confidence", watch=True)
    def _update_map(self):
        self.map = get_map()
        df_aqi = get_dfp_aqi(confidence=self.Confidence)
        add_aqi_predict_circles(self.map, df_aqi)
        self.folium_pane.object = self.map

        
#app = PanelFoliumMapForecast()
#app.view

In [45]:
checkbox = pn.widgets.Checkbox(name='MODIS', value= True, disabled = False )
checkbox

Checkbox(name='MODIS', sizing_mode='stretch_width', value=True)

In [46]:
checkbox2 = pn.widgets.Checkbox(name='VIIRS', value= True, disabled = False )
checkbox2

Checkbox(name='VIIRS', sizing_mode='stretch_width', value=True)

In [47]:
checkbox_group = pn.widgets.CheckBoxGroup(
    name='Checkbox Group', value=['Aqua', 'Tera'], options=['Aqua', 'Tera'], disabled = False,
    inline=True)

checkbox_group

CheckBoxGroup(inline=True, name='Checkbox Group', options=['Aqua', 'Tera'], sizing_mode='stretch_width', value=['Aqua', 'Tera'])

In [48]:
checkbox_group2 = pn.widgets.CheckBoxGroup(
    name='Checkbox Group', value=['S-NPP', 'NOAA 20'], options=['S-NPP', 'NOAA 20'], disabled = False,
    inline=True)

checkbox_group2

CheckBoxGroup(inline=True, name='Checkbox Group', options=['S-NPP', 'NOAA 20'], sizing_mode='stretch_width', value=['S-NPP', 'NOAA 20'])

In [49]:
fig = px.bar(dfh, x='week', y='fire_count', color='year',
       title='Wildfire Hotspot trends in Telengana Nalgonda and Nagarkurnool district')
#fig

## (11) Creating Dashboard

In [50]:
#Layout using Template
template = pn.template.FastListTemplate(
    title='Amrabad Wildfire Prediction', 
    sidebar=[pn.pane.Markdown("## Wildfire Prediction"), 
             pn.pane.Markdown("## Wildfire Detection"),
             pn.pane.Markdown("## Wildfire Trends"),
             #pn.pane.Markdown("#### Wild fire is one of the primary reason for climate change hazard.Track wild fire realtime and forecast wildfire"), 
             #pn.pane.PNG('fire-flame.png', sizing_mode='scale_both'),
             pn.pane.Markdown("## Settings"),
             pn.pane.Markdown("#### Select Satellites & Instruments."),
             pn.Row(checkbox),
             pn.Row(checkbox_group),
             pn.Row(checkbox2),
             pn.Row(checkbox_group2),
             pn.pane.Markdown("#### Select Week Range"),
             pn.Row(week_range_slider)],
    main=[pn.pane.Markdown("# Wildfire Prediction"),
          pn.Row(pn.Column(PanelFoliumMapForecast().view),
                 pn.Column(week_slider,predict_table.panel(width=500))),
          pn.Row(pn.Column(week_vs_conf_scatterplot.panel(width=600), margin=(0,25)), 
                 pn.Column(yaxis_district, week_predict_bar_plot.panel(width=600))),
          pn.pane.Markdown("# Wildfire Detection", align="center"),
          pn.Row(pn.Column(yaxis_duration,
                           PanelFoliumMap().view), 
                 active_fire_table.panel(width=500)),
          pn.pane.Markdown("# Wildfire Trends", align="center"),
          pn.Row(pn.Column(fig))],
    accent_base_color="#88d8b0",
    header_background="#88d8b0",
)
#template.show()
template.servable();